# 範例 14：文件問答系統

讀取文字檔案，讓 AI 回答關於文件內容的問題！

## 學習目標
- 學會讀取和處理文件
- 了解文件切割（Chunking）的概念
- 建立完整的文件問答系統

## 為什麼需要切割文件？
- AI 有輸入長度限制（token limit）
- 小段落更容易精確搜尋
- 可以只傳送相關的部分給 AI，節省資源

## 前置需求
- LM Studio 運行中，Local Server 已啟動
- 安裝 openai 套件：`pip install openai`

## Step 1: 匯入套件

In [ ]:
import os
from openai import OpenAI

# 建立客戶端
client = OpenAI(
    base_url="http://localhost:1234/v1",
    api_key="not-needed"
)

## Step 2: 文件讀取函數

In [ ]:
def read_document(file_path):
    """
    讀取文件內容

    參數：
        file_path: 文件路徑

    回傳：
        文件內容
    """
    with open(file_path, "r", encoding="utf-8") as f:
        return f.read()

## Step 3: 文件切割函數

將長文件切割成小段落，這是 RAG 系統中很重要的技術！

In [ ]:
def chunk_document(text, chunk_size=500, overlap=50):
    """
    將長文件切割成小段落

    為什麼要切割？
    - AI 有輸入長度限制
    - 小段落更容易精確搜尋
    - 可以只傳送相關的部分給 AI

    參數：
        text: 文件內容
        chunk_size: 每段的字數
        overlap: 段落之間重疊的字數（避免資訊被切斷）
    """
    chunks = []
    start = 0

    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start = end - overlap  # 重疊部分，確保資訊連續

    return chunks

In [ ]:
# 測試文件切割
sample_text = "A" * 100 + "B" * 100 + "C" * 100 + "D" * 100 + "E" * 100  # 500 字
chunks = chunk_document(sample_text, chunk_size=150, overlap=30)

print(f"原始文字長度：{len(sample_text)}")
print(f"切割成 {len(chunks)} 個段落：")
for i, chunk in enumerate(chunks):
    print(f"  段落 {i+1}：長度 {len(chunk)}，內容：{chunk[:20]}...{chunk[-20:]}")

## Step 4: 建立文件問答系統類別

In [ ]:
class DocumentQA:
    """
    文件問答系統類別
    """

    def __init__(self):
        self.client = OpenAI(
            base_url="http://localhost:1234/v1",
            api_key="not-needed"
        )
        self.chunks = []
        self.document_loaded = False

    def load_document(self, file_path):
        """從檔案載入文件"""
        text = read_document(file_path)
        self.chunks = chunk_document(text)
        self.document_loaded = True
        print(f"已載入文件：{file_path}")
        print(f"共 {len(self.chunks)} 個段落")

    def load_text(self, text):
        """直接載入文字"""
        self.chunks = chunk_document(text)
        self.document_loaded = True
        print(f"已載入文字，共 {len(self.chunks)} 個段落")

    def find_relevant_chunks(self, question, top_k=3):
        """
        找出與問題相關的段落（簡單的關鍵字匹配）
        """
        scored_chunks = []

        # 將問題拆成關鍵字
        keywords = question.lower().split()

        for chunk in self.chunks:
            score = 0
            chunk_lower = chunk.lower()

            # 計算每個關鍵字出現的次數
            for keyword in keywords:
                if keyword in chunk_lower:
                    score += chunk_lower.count(keyword)

            scored_chunks.append((score, chunk))

        # 排序並返回最相關的段落
        scored_chunks.sort(reverse=True)
        return [chunk for score, chunk in scored_chunks[:top_k] if score > 0]

    def ask(self, question):
        """
        提問並獲得回答
        """
        if not self.document_loaded:
            return "請先載入文件！"
        
        # 找出相關段落
        relevant_chunks = self.find_relevant_chunks(question)

        if not relevant_chunks:
            return "抱歉，我在文件中找不到相關資訊。"

        # 建立上下文
        context = "\n---\n".join(relevant_chunks)

        prompt = f"""你是一個文件問答助手。請根據以下文件內容回答問題。
如果文件中沒有相關資訊，請誠實說明。

文件內容：
{context}

問題：{question}

請用繁體中文回答："""

        response = self.client.chat.completions.create(
            model="gpt-oss-120b",
            messages=[{"role": "user", "content": prompt}]
        )

        return response.choices[0].message.content
    
    def get_stats(self):
        """取得文件統計資訊"""
        if not self.document_loaded:
            return "尚未載入文件"
        
        total_chars = sum(len(chunk) for chunk in self.chunks)
        return {
            "段落數": len(self.chunks),
            "總字數": total_chars,
            "平均段落長度": total_chars // len(self.chunks) if self.chunks else 0
        }

## Step 5: 載入範例文件

In [ ]:
# 建立問答系統
qa = DocumentQA()

# 範例文字（模擬一份關於 AI 的文件）
sample_document = """
人工智慧（Artificial Intelligence，簡稱 AI）是電腦科學的一個分支，
致力於創造能夠執行通常需要人類智慧的任務的機器。這些任務包括學習、
推理、問題解決、感知和語言理解。

機器學習是人工智慧的一個子領域，專注於開發能夠從資料中學習的演算法。
深度學習是機器學習的一個分支，使用多層神經網路來處理複雜的資料模式。

自然語言處理（NLP）是 AI 的另一個重要領域，讓電腦能夠理解、解釋和
生成人類語言。ChatGPT 就是一個著名的 NLP 應用。

AI 的應用非常廣泛，包括：
- 語音助手（如 Siri、Alexa）
- 自動駕駛汽車
- 醫療診斷輔助
- 推薦系統（如 Netflix、YouTube）
- 遊戲 AI

AI 的發展歷史可以追溯到 1950 年代。1956 年的達特茅斯會議被認為是
AI 研究的正式開端。此後，AI 經歷了多次「AI 寒冬」和復興。

近年來，由於計算能力的提升和大數據的普及，AI 取得了突破性的進展。
特別是 2017 年 Transformer 架構的出現，徹底改變了自然語言處理領域。

AI 的倫理問題也越來越受到關注，包括：
- 隱私保護
- 演算法偏見
- 就業影響
- AI 安全性
"""

# 載入文字
qa.load_text(sample_document)

# 顯示統計
print(f"\n文件統計：{qa.get_stats()}")

## Step 6: 測試文件問答

In [ ]:
# 問題 1
question1 = "什麼是機器學習？"
print(f"問題：{question1}")
print("-" * 50)
print(f"回答：{qa.ask(question1)}")

In [ ]:
# 問題 2
question2 = "AI 有哪些應用？"
print(f"問題：{question2}")
print("-" * 50)
print(f"回答：{qa.ask(question2)}")

In [ ]:
# 問題 3
question3 = "AI 發展有什麼倫理問題？"
print(f"問題：{question3}")
print("-" * 50)
print(f"回答：{qa.ask(question3)}")

In [ ]:
# 問題 4（文件中沒有的資訊）
question4 = "量子電腦是什麼？"
print(f"問題：{question4}")
print("-" * 50)
print(f"回答：{qa.ask(question4)}")

## Step 7: 進階功能 - 顯示來源段落

In [ ]:
class DocumentQAWithSources(DocumentQA):
    """
    增強版：會顯示回答的來源段落
    """
    
    def ask_with_sources(self, question):
        """
        提問並顯示來源
        """
        if not self.document_loaded:
            return "請先載入文件！", []
        
        # 找出相關段落
        relevant_chunks = self.find_relevant_chunks(question)

        if not relevant_chunks:
            return "抱歉，我在文件中找不到相關資訊。", []

        # 建立上下文
        context = "\n---\n".join(relevant_chunks)

        prompt = f"""你是一個文件問答助手。請根據以下文件內容回答問題。
如果文件中沒有相關資訊，請誠實說明。

文件內容：
{context}

問題：{question}

請用繁體中文回答："""

        response = self.client.chat.completions.create(
            model="gpt-oss-120b",
            messages=[{"role": "user", "content": prompt}]
        )

        return response.choices[0].message.content, relevant_chunks

In [ ]:
# 測試顯示來源
qa_sources = DocumentQAWithSources()
qa_sources.load_text(sample_document)

question = "什麼是 NLP？"
answer, sources = qa_sources.ask_with_sources(question)

print(f"問題：{question}")
print("=" * 50)
print(f"回答：{answer}")
print("\n參考段落：")
for i, source in enumerate(sources, 1):
    print(f"\n[段落 {i}]")
    print(source[:200] + "..." if len(source) > 200 else source)

## 文件切割策略

### 不同的切割方式：

1. **固定長度切割**（本範例使用）
   - 優點：簡單、一致
   - 缺點：可能切斷句子

2. **按段落切割**
   - 優點：保持語義完整
   - 缺點：段落長度不一

3. **按句子切割**
   - 優點：句子完整
   - 缺點：單一句子可能資訊不足

4. **智慧切割**（使用 AI）
   - 優點：語義完整
   - 缺點：成本較高

## 練習區

In [ ]:
# 試著載入你自己的文件
my_qa = DocumentQA()

# 方法 1：載入文字
my_text = """
在這裡貼上你想要問答的文字...
"""
# my_qa.load_text(my_text)

# 方法 2：載入檔案
# my_qa.load_document("your_file.txt")

# 提問
# print(my_qa.ask("你的問題"))

## 重點回顧

1. **文件切割**：將長文件分成小段落
   - `chunk_size`：每段的大小
   - `overlap`：段落間的重疊（避免切斷資訊）

2. **相關性搜尋**：找出與問題相關的段落

3. **問答流程**：
   - 載入文件 → 切割 → 搜尋相關段落 → AI 回答

4. **來源追蹤**：顯示回答的依據，增加可信度

## 下一步

接下來我們將學習 Fine-Tuning（微調）的相關技術！